# Plot data

Using functions from https://github.com/sanil72900/argo_notebook/blob/main/lesson_one.ipynb

## To use, need to install ipywidgets to environment

## Import packages

In [1]:
#data visualization
import matplotlib.pylab as plt
from matplotlib import colors
%matplotlib inline

#used for map projections
import cartopy.crs as ccrs

#widgets for user interaction
import ipywidgets as widgets

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'ipywidgets'

## Plot Data Class

In [ ]:
class PlotData:
    
    def __init__(self):
        pass
    
    @classmethod
    def plot_pmesh(self, selectionDf, figsize=(12,12), shrinkcbar=.1, delta_lon=75, delta_lat=75, \
                   map_proj=ccrs.PlateCarree(), xlims=None):
        
        df = selectionDf
        
        fig = plt.figure(figsize=figsize)

        #collect lon, lat values to create coordinates
        x = df['lon'].values                   
        y = df['lat'].values
        
        points = map_proj.transform_points(ccrs.Geodetic(), x, y)
        x = points[:, 0]
        y = points[:, 1]

        #plot coordinates
        map_proj._threshold /= 100
        ax = plt.axes(projection=map_proj, xlabel='long', ylabel='lats')
        ax.set_global()
        plt.title('Spatial selection of Argo profile locations on map')
        sct = plt.scatter(x, y, c='r', s=15,zorder=3) 

        #specify boundaries for map plot
        if not xlims:
            xlims = [df['lon'].max() + delta_lon, df['lon'].min()- delta_lon]
        ax.set_xlim(min(xlims), max(xlims))
        ax.set_ylim(min(df['lat']) - delta_lat, max(df['lat']) + delta_lat)

        ax.coastlines(zorder=1)
        ax.stock_img()
        ax.gridlines()
        return fig
    
    @classmethod
    def create_prop_plot_widget(self, selectionDf):
    
        def property_plot_selection(self, x_col, y_col, z_col):

            print('Recalculating....')

            fig, ax = plt.subplots()    

            cmap_iter = len(selectionDf.groupby([z_col]))

            i = 0

            #create color map array 
            colors = plt.cm.jet(np.linspace(0, 1, cmap_iter))

            #group lines by z-variable
            for key, group in selectionDf.groupby([z_col]):
                ax = group.plot(ax=ax, kind='line', x=x_col, y=y_col, color = colors[i], label=key, figsize=(15,10), alpha=0.5)
                i += 1

            #inverse y-axis if pressure is chosen
            if y_col == 'pres':
                ax.set_ylim(ax.get_ylim()[::-1])

            #generate plot
            ax.set_title(unit_dict[x_col].split(' ')[0]+ ' vs. ' +unit_dict[y_col].split(' ')[0] + ' Argo profiles')
            ax.set_xlabel(unit_dict[x_col])
            ax.set_ylabel(unit_dict[y_col])
            
        #options for drop-drown menus
        x = [('Temperature', 'temp'),('Pressure', 'pres'),('Salinity', 'psal')]
        y = [('Pressure', 'pres'),('Temperature', 'temp'),('Salinity', 'psal'),('Longitude', 'lon'),('Latitude', 'lat')]
        z = [('Profile ID', 'profile_id'),('Day', 'day'), ('Year', 'year'), ('Month', 'month')]

        #initialize widgets
        x_drop = widgets.Dropdown(options=x, val='Temperature', description='X-variable', disabled=False)
        y_drop = widgets.Dropdown(options=y, val='Pressure', description='Y-variable', disabled=False)
        z_drop = widgets.Dropdown(options=z, val='Profile ID', description='Z-variable', disabled=False)
        widgets.VBox([x_drop, y_drop, z_drop])
        
        plot_shape = widgets.interact(property_plot_selection, x_col=x_drop, y_col=y_drop, z_col=z_drop)

        return plot_shape
    
    def create_scatter_plot_widget(self, selectionDf):
        
        def scatter_plot_selection(x_col, z_col, num_ints, y_col='pres'):
            
            print('Recalculating....')
            fig, ax = plt.subplots()
            fig.set_size_inches(18.5, 10.5)

            c_step = (max(selectionDf[z_col])-min(selectionDf[z_col]))/num_ints
            
            cmap = plt.cm.rainbow
            
            norm = colors.BoundaryNorm(np.arange(min(selectionDf[z_col]), max(selectionDf[z_col]), c_step), cmap.N)

            #group lines by z-variable
            plt.scatter(selectionDf[x_col], selectionDf[y_col], c=selectionDf[z_col], cmap=cmap, s=50, norm=norm)
            plt.colorbar(ticks=np.linspace(max(selectionDf[z_col])+1, min(selectionDf[z_col]), num_ints+1))

            #inverse y-axis for pressure
            ax.set_ylim(ax.get_ylim()[::-1])

            #generate plot
            ax.set_title(unit_dict[x_col].split(' ')[0]+ ' vs. Pressure Argo profiles')
            ax.set_xlabel(unit_dict[x_col])
            plt.xticks([])
            ax.set_ylabel('Pressure [dbar]')
            
        #options for drop-drown menus
        x_option = [('Time', 'date'),('Longitude', 'lon'),('Latitude', 'lat')]
        z_option = [('Temperature', 'temp'),('Salinity', 'psal')]

        x_pick = widgets.Dropdown(options=x_option, val='Time', description='X-variable', disabled=False)
        z_pick = widgets.Dropdown(options=z_option, val='Temperature', description='Z-variable', disabled=False)

        widgets.VBox([x_pick, z_pick])
        
        scatter_shape = widgets.interact(scatter_plot_selection, x_col=x_pick, z_col=z_pick, num_ints=10)
        
        return scatter_shape
    